In [ ]:
import pandas as pd
data_file = '../data/iris.data'
colnames = [
    'sepal length',
    'sepal width',
    'petal length',
    'petal width',
    'species'
]
iris = pd.read_csv(data_file, names=colnames, index_col=False)
iris.head()

# HoloMap

A container for multi-dimensional data for use in exploring variable space. A HoloMap enumerates specific values for each variable and contains explicit plotting data for each combination. The whole thing is encoded as a javascript-readable object and thus will work in static versions of this notebook. 

In [ ]:
import numpy as np
import holoviews as hv
from holoviews import opts
hv.notebook_extension('bokeh')
opts.defaults(opts.Curve(line_width=1))

In [ ]:
def fm_modulation(f_carrier=220, f_mod=220, mod_index=1, length=0.1, sampleRate=2000):
    sampleInc = 1.0/sampleRate
    x = np.arange(0,length, sampleInc)
    y = np.sin(2*np.pi*f_carrier*x + mod_index*np.sin(2*np.pi*f_mod*x))
    return hv.Curve((x, y), 'Time', 'Amplitude')

In [ ]:
f_carrier = np.linspace(20, 60, 3)
f_mod = np.linspace(20, 100, 5)

curve_dict = {(fc, fm): fm_modulation(fc, fm) for fc in f_carrier for fm in f_mod}

kdims = [hv.Dimension(('f_carrier', 'Carrier frequency'), default=40),
         hv.Dimension(('f_mod', 'Modulation frequency'), default=60)]
holomap = hv.HoloMap(curve_dict, kdims=kdims)
holomap.opts(opts.Curve(width=600))

# DynamicMap

A container for exploring high-dimensional data that dynamically generates the data on-demand in order to save memory.

Because data is generated by the Python kernel on-demand, these examples will not work properly in static copies of this notebook. A live Jupyter instance is required.

In [ ]:
import numpy as np
import holoviews as hv
from holoviews import opts

hv.extension('matplotlib')

In [ ]:
xvals  = np.linspace(-4, 0, 202)
yvals  = np.linspace(4, 0, 202)
xs, ys = np.meshgrid(xvals, yvals)

# a function that generates a data set based on input variable values
def waves_image(alpha, beta):
    return hv.Image(np.sin(((ys/alpha)**alpha+beta)*xs))

# snapshots exploring the variable space
waves_image(1,0) + waves_image(1,4)

In [ ]:
# like everything else, the DynamicMap is a composable element
dmap = hv.DynamicMap(waves_image, kdims=['alpha', 'beta'])
dmap

In [ ]:
# Two ways to generate data from the DynamicMap
dmap[1, 2] + dmap.select(alpha=1, beta=2)

In [ ]:
# Use redim to specify ranges (alternatively this could be done with hv.Dimension())
dmr = dmap.redim.range(alpha=(1, 5.0), beta=(1, 6.0))
dmr

In [ ]:
# Alternately use redim.values to specify a list of values
dmap.redim.values(alpha=[1, 2, 3], beta=[0.1, 1.0, 2.5])

# Streams

Where `DynamicMaps` let us explore data calculated on-the-fly from defined functions, `Streams` let us explore any sort of data and interact with it.

Again, the Python kernel is providing data in response to requests from either your browser or subsequent Python code. These examples will not work in static versions of this notebook.

In [ ]:
hv.extension('bokeh')

opts.defaults(
    opts.Area(fill_color='cornsilk', line_width=2,
              line_color='black'),
    opts.Ellipse(bgcolor='white', color='black'),
    opts.HLine(color='red', line_width=2),
    opts.Image(cmap='viridis'),
    opts.Path(bgcolor='white', color='black', line_dash='dashdot',
              padding=0.1, show_grid=False),
    opts.VLine(color='red', line_width=2))

In [ ]:
# For this example, we first create a Dynamic Map
lin = np.linspace(-np.pi,np.pi,300)

def lissajous(t, a=3, b=5, delta=np.pi/2.):
    return (np.sin(a * t + delta), np.sin(b * t))

def lissajous_crosshair(t, a=3, b=5, delta=np.pi/2):
    (x,y) = lissajous(t,a,b,delta)
    return hv.VLine(x) * hv.HLine(y)

crosshair = hv.DynamicMap(lissajous_crosshair, kdims='t').redim.range(t=(-3.,3.))

path = hv.Path(lissajous(lin))

path * crosshair

In [ ]:
from holoviews.streams import Stream, param
# Define Time as a new subclass to Stream
Time = Stream.define('Time', t=param.Number(default=0.0, doc='A time parameter'))

# create an instance of Time
time = Time(t=np.pi/4)

dmap = hv.DynamicMap(lissajous_crosshair, streams=[time])
path * dmap + path * lissajous_crosshair(t=np.pi/4)

Now we have the `DynamicMap` backed by a `Stream` next to a static version, both have the same initial value of t. 

No widgets are linked to the streaming version, but we can still update the value of t by calling the following:

In [ ]:
dmap.event(t=0.2)

# Mixing Streams and Key Dimension Widgets

In [ ]:
# 400 points from -3 to 3
xs = np.linspace(-3, 3, 400)

def function(xs, time):
    "Some time varying function"
    return np.exp(np.sin(xs+np.pi/time))

def integral(limit, time):
    curve = hv.Curve((xs, function(xs, time)))[limit:]
    area  = hv.Area ((xs, function(xs, time)))[:limit]
    summed = area.dimension_values('y').sum() * 0.015  # Numeric approximation
    return (area * curve * hv.VLine(limit) * hv.Text(limit + 0.5, 2.0, '%.2f' % summed))

Time = Stream.define('Time', time=1.0)

# create a DynamicMap with both a stream and kdim
dmap=hv.DynamicMap(
    integral,
    kdims='limit',
    streams=[Time()]).redim.range(limit=(-3,2))
dmap

In [ ]:
dmap.event(time=5)

# Generators and Streams

Streams offer us a way to update visualizations with new data. 

One thing we could do is use a common feature of Python called a `generator` to provide subsets of data in series with each call to the generator iterator. 

In [ ]:
# generators are functions that call yield to return a value
# they return an iterator which will return each value yielded and
# continue execution after the yield function until the end is reached
def sample_distributions(samples=10, tol=0.04):
    np.random.seed(42)
    while True:
        gauss1 = np.random.normal(size=samples)
        gauss2 = np.random.normal(size=samples)
        data = (['A']*samples + ['B']*samples, np.hstack([gauss1, gauss2]))
        yield hv.BoxWhisker(data, 'Group', 'Value')
        samples+=1

# create our iterator
sample_generator = sample_distributions()

In [ ]:
# Define a stream with no parameters since generators cannot take any
dmap = hv.DynamicMap(sample_generator, streams=[Stream.define('Next')()])
dmap

In [ ]:
# update dmap 10x per second to step through all the data sets
dmap.periodic(0.1, 1000, timeout=3)